### 创建数据集

In [93]:
# 创建数据集
with open('/home/fonttian/Data/Scrapy/example/example1_1.html','r') as f:
    text = f.read()

print(text)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
<h1>Hello World</h1>
<h1>Hello World</h1>
<h1>Hello World</h1>

<ul>
    <li>C++</li>
    <li>JAVA</li>
    <li>Scala</li>
    <li>Python</li>
    <li>R</li>
</ul>
</body>
</html>


In [12]:
text2 = '\
    <body>\
    <h1>Hello World</h1>\
    <h1>Hello World<h2>23333</h2></h1>\
    <h1>Hello World</h1>\
    <ul>\
        <li>C++</li>\
        <li>JAVA</li>\
        <li>Scala</li>\
        <li>Python</li>\
        <li>R</li>\
    </ul>\
    </body>\
    </html>'

### 创建对象

In [13]:
# 在构造Seletor时,将页面以字符串的形式传给Selector

from scrapy.selector import Selector

selector = Selector(text = text2)
selector

<Selector xpath=None data='<html><body>    <h1>Hello World</h1>    '>

In [4]:
# 使用Response对象构造Selector对象,然后通过参数传递给Selector

from scrapy.http import HtmlResponse
    
response = HtmlResponse(url = 'http://blog.csdn.net/fontthrone/article/category/6877815',body = text,encoding = 'utf8')
selector = Selector(response=response)
selector

<Selector xpath=None data='<html lang="en">\n<head>\n    <meta charse'>

### 选中数据

In [14]:
selector_list = selector.xpath('//h1') # // 选中所有,会返回一个Selectorlist对象
selector_list # SelectorList中的每一个对象都是一个Selector

[<Selector xpath='//h1' data='<h1>Hello World</h1>'>,
 <Selector xpath='//h1' data='<h1>Hello World<h2>23333</h2></h1>'>,
 <Selector xpath='//h1' data='<h1>Hello World</h1>'>]

In [22]:
for sel in selector_list:
    print('     ',sel.xpath('./text()'))
    print('str :',str(sel.xpath('./text()'))) # 注意是不能这样提取数据的 

      [<Selector xpath='./text()' data='Hello World'>]
str : [<Selector xpath='./text()' data='Hello World'>]
      [<Selector xpath='./text()' data='Hello World'>]
str : [<Selector xpath='./text()' data='Hello World'>]
      [<Selector xpath='./text()' data='Hello World'>]
str : [<Selector xpath='./text()' data='Hello World'>]


In [94]:
selector_list.xpath('./text()')

[<Selector xpath='./text()' data='Hello World'>,
 <Selector xpath='./text()' data='Hello World'>,
 <Selector xpath='./text()' data='Hello World'>]

In [107]:
selector.xpath('//ul').css('li').xpath('./text()')

[<Selector xpath='./text()' data='C++'>,
 <Selector xpath='./text()' data='JAVA'>,
 <Selector xpath='./text()' data='Scala'>,
 <Selector xpath='./text()' data='Python'>,
 <Selector xpath='./text()' data='R'>]

In [105]:
selector.xpath('./ul')

[]

这是为什么呢?

In [103]:
selector

<Selector xpath=None data='<html><body>    <h1>Hello World</h1>    '>

In [112]:
selector.xpath('/html/body/ul')

[<Selector xpath='/html/body/ul' data='<ul>        <li>C++</li>        <li>JAVA'>]

### 提取数据

有四种方式
 1. extract()
 2. re()
 3. extract_first() (SelectorList专有)
 4. re_first() (SelectorList专有)
 

In [34]:
sl_li = selector.xpath('//li') # SelectorList对象
sl_li

[<Selector xpath='//li' data='<li>C++</li>'>,
 <Selector xpath='//li' data='<li>JAVA</li>'>,
 <Selector xpath='//li' data='<li>Scala</li>'>,
 <Selector xpath='//li' data='<li>Python</li>'>,
 <Selector xpath='//li' data='<li>R</li>'>]

In [40]:
sl_li[0].extract()
# sl_li.extract_first() # 拥有同样的效果

'<li>C++</li>'

In [44]:
# sl_li_text = sl_li.xpath('./text()')
# sl_li = selector.xpath('//li/text()') # SelectorList对象,效果一样
sl_li = selector.xpath('//li').xpath('./text()') # SelectorList对象,效果一样
sl_li_text[0].extract()

'C++'

In [46]:
sl_li = selector.xpath('//li').xpath('./text()') # 实现方式不同,对象本质不同
sl_li

[<Selector xpath='./text()' data='C++'>,
 <Selector xpath='./text()' data='JAVA'>,
 <Selector xpath='./text()' data='Scala'>,
 <Selector xpath='./text()' data='Python'>,
 <Selector xpath='./text()' data='R'>]

In [48]:
sl_li.extract()

['C++', 'JAVA', 'Scala', 'Python', 'R']

In [57]:
for sel in selector_list:
    print('\n     ',sel.xpath('./text()'))
#     print('str :',str(sel.xpath('./text()'))) # 注意是不能这样提取数据的
    print('\n     ',sel.xpath('./text()').extract()[0])


      [<Selector xpath='./text()' data='Hello World'>]

      Hello World

      [<Selector xpath='./text()' data='Hello World'>]

      Hello World

      [<Selector xpath='./text()' data='Hello World'>]

      Hello World


In [58]:
selector_list.extract_first()

'<h1>Hello World</h1>'

In [59]:
selector_list.extract()

['<h1>Hello World</h1>',
 '<h1>Hello World<h2>23333</h2></h1>',
 '<h1>Hello World</h1>']

In [61]:
selector_list[0].extract()

'<h1>Hello World</h1>'

In [62]:
# re

text3 = '\
    <ul>\
        <li>C++<b>课时32小时</b></li>\
        <li>JAVA<b>课时32小时</b></li>\
        <li>Scala<b>课时32小时</b></li>\
        <li>Python<b>课时32小时</b></li>\
        <li>R<b>课时32小时</b></li>\
    </ul>'


selector3 = Selector(text=text3)

In [64]:
selector3.xpath('//li/b/text()')

[<Selector xpath='//li/b/text()' data='课时32小时'>,
 <Selector xpath='//li/b/text()' data='课时32小时'>,
 <Selector xpath='//li/b/text()' data='课时32小时'>,
 <Selector xpath='//li/b/text()' data='课时32小时'>,
 <Selector xpath='//li/b/text()' data='课时32小时'>]

In [69]:
selector3.xpath('//li/b/text()').extract()

['课时32小时', '课时32小时', '课时32小时', '课时32小时', '课时32小时']

In [71]:
type(selector3.xpath('//li/b/text()'))

scrapy.selector.unified.SelectorList

In [72]:
type(selector3.xpath('//li/b/text()').extract())

list

In [74]:
selector3.xpath('//li/b/text()').re('\d+')

['32', '32', '32', '32', '32']

In [76]:
type(selector3.xpath('//li/b/text()').re('\d+'))

list

## Response 的内置Selector
TextResponse的源码,会自动创建Selector对象

```python

class TextResponse(Response):

    @property
    def selector(self):
        from scrapy.selector import Selector
        if self._cached_selector is None:
            self._cached_selector = Selector(self)
        return self._cached_selector

```

In [81]:
response = HtmlResponse(url='http://blog.csdn.net/fontthrone/article/category/6877815',body=text3,encoding='utf8')
Selector_response = response.selector
Selector_response

<Selector xpath=None data='<html><body><ul>        <li>C++<b>课时32小时'>

In [86]:
Selector_response.re('\d+')

['32', '32', '32', '32', '32']

In [87]:
response.re('\b+') # 不行

AttributeError: 'HtmlResponse' object has no attribute 're'

In [89]:
response.xpath('//li')

[<Selector xpath='//li' data='<li>C++<b>课时32小时</b></li>'>,
 <Selector xpath='//li' data='<li>JAVA<b>课时32小时</b></li>'>,
 <Selector xpath='//li' data='<li>Scala<b>课时32小时</b></li>'>,
 <Selector xpath='//li' data='<li>Python<b>课时32小时</b></li>'>,
 <Selector xpath='//li' data='<li>R<b>课时32小时</b></li>'>]

In [90]:
response.css('li')

[<Selector xpath='descendant-or-self::li' data='<li>C++<b>课时32小时</b></li>'>,
 <Selector xpath='descendant-or-self::li' data='<li>JAVA<b>课时32小时</b></li>'>,
 <Selector xpath='descendant-or-self::li' data='<li>Scala<b>课时32小时</b></li>'>,
 <Selector xpath='descendant-or-self::li' data='<li>Python<b>课时32小时</b></li>'>,
 <Selector xpath='descendant-or-self::li' data='<li>R<b>课时32小时</b></li>'>]